In [1]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import StandardScaler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer

In [2]:
if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("NaiveBayes")\
        .getOrCreate()

In [3]:
dataset = spark.read.csv("diabetes.csv",header=True)

In [4]:
dataset.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|  31|                   0.248| 26|      1|


In [5]:
dataset.printSchema()

root
 |-- Pregnancies: string (nullable = true)
 |-- Glucose: string (nullable = true)
 |-- BloodPressure: string (nullable = true)
 |-- SkinThickness: string (nullable = true)
 |-- Insulin: string (nullable = true)
 |-- BMI: string (nullable = true)
 |-- DiabetesPedigreeFunction: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Outcome: string (nullable = true)



In [6]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast("float").alias(c) for c in dataset.columns))

In [7]:
new_data.printSchema()

root
 |-- Pregnancies: float (nullable = true)
 |-- Glucose: float (nullable = true)
 |-- BloodPressure: float (nullable = true)
 |-- SkinThickness: float (nullable = true)
 |-- Insulin: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- DiabetesPedigreeFunction: float (nullable = true)
 |-- Age: float (nullable = true)
 |-- Outcome: float (nullable = true)



In [8]:
from pyspark.sql.functions import col, count, isnan, when
#checking for null ir nan type values in our columns
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|          0|      0|            0|            0|      0|  0|                       0|  0|      0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [23]:
cols=new_data.columns
cols.remove("Outcome")
assembler = VectorAssembler(inputCols=cols,outputCol="features")
# Now let us use the transform method to transform our dataset
data=assembler.transform(new_data)
data = data.select("features",'Outcome')

In [24]:
(train, test) = data.randomSplit([0.7, 0.3])

In [25]:
naive_bayes = NaiveBayes(featuresCol='features',labelCol='Outcome',smoothing=1.0)

In [26]:
model = naive_bayes.fit(train) 

In [27]:
# select example rows to display.
predictions = model.transform(test)

In [28]:
predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|(8,[0,1,6,7],[2.0...|    0.0|[-143.83567904225...|[0.34076013715848...|       1.0|
|(8,[0,1,6,7],[10....|    1.0|[-246.02480623093...|[0.04183600349677...|       1.0|
|(8,[1,5,6,7],[119...|    1.0|[-266.73892610504...|[0.74238623745696...|       0.0|
|(8,[1,5,6,7],[145...|    1.0|[-343.41841982861...|[0.82114305732646...|       0.0|
|(8,[1,5,6,7],[167...|    1.0|[-338.19925421681...|[0.54292199945573...|       0.0|
|(8,[1,6,7],[94.0,...|    0.0|[-164.71757704226...|[0.36543019856782...|       1.0|
|[0.0,67.0,76.0,0....|    0.0|[-412.92025298417...|[0.99999996307298...|       0.0|
|[0.0,86.0,68.0,32...|    0.0|[-439.28761904950...|[0.99999999530970...|       0.0|
|[0.0,95.0,64.0,39...|    0.0|[-643.27046136234...|[0.70105869360027...|    

In [31]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
labelCol="Outcome", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)


In [32]:
print ("Accuracy",accuracy)

Accuracy 0.589041095890411


In [33]:
spark.stop()